In [24]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
import csv
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.tree import DecisionTreeRegressor

In [2]:
class classGetData():
    def __init__(self,fileloc):
        self.loc = fileloc
        print('Init...')
        

    def readCSV(self):
        #data1 = open('./data/test.csv')
        print('Reading...')
        file_open = open(self.loc)
        datareader = csv.reader(file_open,delimiter= ',')
        data_arr = []
        for row in datareader:
            data_arr.append(row)

        arr = np.array(data_arr)
        #arr = arr[1:,:]
        print('Done...')
        return arr

In [3]:
def read_data(loc_in):
    a = classGetData(loc_in)
    data1 = a.readCSV()
    return data1

In [4]:
loc = './forestfires.csv'
raw = read_data(loc)
raw_data = raw[1:,:]
title = raw[0,:]

title = np.delete(title,[2,3],0) #remove month and date

X = raw_data[:,:-1]
X = np.delete(X,[2,3],1) #remove month and date
y = raw_data[:,-1]

X = X.astype(float)
y = y.astype(float)

# binary case
y[y.nonzero()] = 1

Init...
Reading...
Done...


In [5]:
print(title)

['X' 'Y' 'FFMC' 'DMC' 'DC' 'ISI' 'temp' 'RH' 'wind' 'rain' 'area']


# Training Validation Test split

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_t, X_train_v, y_train_t, y_train_v = train_test_split(X_train, y_train, test_size=0.2, random_state=55)



In [7]:
def evaluation(y_predict, y_test):
    print("Mean squared error: %.2f" % mean_squared_error(y_test, y_predict))
    # Explained variance score: 1 is perfect prediction
    print('Variance score: %.2f' % r2_score(y_test, y_predict))

# Linear regression

In [8]:
lin_model=LinearRegression().fit(X_train_t,y_train_t)

y_predict = lin_model.predict(X_test)

evaluation(y_predict.reshape(-1,1), y_test.reshape(-1,1))

Mean squared error: 0.25
Variance score: 0.00


# Logestic regression

In [9]:
clf = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial').fit(X_train_t,y_train_t)

y_log_predict = clf.predict(X_test)

evaluation(y_predict, y_test)

Mean squared error: 0.25
Variance score: 0.00


D:\Program\Anaconda\envs\py36\lib\site-packages\sklearn\linear_model\logistic.py:757: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


# Decision Tree Classification

In [10]:
import graphviz

In [11]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train_t, y_train_t)

y_dec_predict = clf.predict(X_test)



In [21]:
dot_data = tree.export_graphviz(clf, out_file=None,feature_names=title[:-1],
                                class_names=['0','1'], filled=True, 
                                rounded=True,special_characters=True) 
graph = graphviz.Source(dot_data) 
graph.render("decision") 
graphviz.Source(dot_data).view()

'Source.gv.pdf'

In [22]:
'''dot_data = tree.export_graphviz(clf, out_file=None) 
graph = graphviz.Source(dot_data) 
graph.render("decision") 
graphviz.Source(dot_data).view()'''

'dot_data = tree.export_graphviz(clf, out_file=None) \ngraph = graphviz.Source(dot_data) \ngraph.render("decision") \ngraphviz.Source(dot_data).view()'

# SVM - classification

In [41]:
from sklearn import svm

clf = svm.SVC(gamma='scale',decision_function_shape='ovo').fit(X_train_t, y_train_t)
svm_predict = clf.predict(X_test)
evaluation(y_test, svm_predict)
print('\n')
# Kernels
kern = ['linear', 'rbf', 'sigmoid']

for k in kern:
    clf = svm.SVC(gamma='scale',decision_function_shape='ovo', kernel = k).fit(X_train_t, y_train_t)
    svm_predict = clf.predict(X_test)
    print('kernel ', k)
    evaluation(y_test, svm_predict)
    print('\n')
    
  

#clf = svm.SVC(gamma='scale').fit(X_train_t, y_train_t)



Mean squared error: 0.38
Variance score: -0.57


kernel  linear
Mean squared error: 0.48
Variance score: -0.96


kernel  rbf
Mean squared error: 0.38
Variance score: -0.57


kernel  sigmoid
Mean squared error: 0.49
Variance score: 0.00




# SVM - Regression

In [36]:
from sklearn import svm

clf = svm.SVR(gamma='scale').fit(X_train_t, y_train_t)
#clf = svm.SVC(gamma='scale').fit(X_train_t, y_train_t)

svm_predict = clf.predict(X_test)

evaluation(y_test, svm_predict)

Mean squared error: 0.28
Variance score: -1.97
